In [31]:
import torch
import torch.nn as nn, optim
from transformers import *
import pandas as pd
import ast
import copy
import os
from opencc import OpenCC

# Model
class bertwwm(nn.Module):
    def __init__(self):
        super(bertwwm,self).__init__()
        self.bert_model = BertModel.from_pretrained("hfl/chinese-bert-wwm")
        self.tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm")
        self.bi_decoder = nn.Sequential(
            nn.Linear(768,512)
            ,nn.Dropout(0.1)
            ,nn.Linear(512,256)
            ,nn.Linear(256,64)
            ,nn.Linear(64,1)
            ,nn.Sigmoid()
        )
    def forward(self,passage):
        # input a string
        input_ids = torch.tensor([self.tokenizer.encode("passage")])
        all_hidden_states, all_attentions = self.bert_model(input_ids)[-2:]
        binary = self.bi_decoder(all_attentions).squeeze()
        return binary

In [ ]:
def test(model,device):
    path = './test.csv'
    data = pd.load_csv(path)
    data =  data.drop(columns=['hyperlink','content','html','clean_content']).values.tolist()
    model.eval()
    loss, size = 0, 0
    criterion = nn.BCELoss()
    for i,n_id,name,content in enumerate(data):
        
        if len(ast.literal_eval(name)) is 0:    
            label = torch.Tensor([0]).to(gpu)
        else:
            label = torch.Tensor([1]).to(gpu)
        size +=1
        loss += criterion(model(content),label)
    loss /= size
    
    return loss

In [ ]:
# lr  : learning rate
# w_d :weight_decay

def train(data,lr_rate=0.001 ,w_d=5e-5 ,device):
  
    model = bertwwm().to(device)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adadelta(parameters, lr=lr_rate, weight_decay=w_d)
    criterion = nn.BCELoss()
    
    cc = OpenCC('tw2sp')
    for i,n_id,name,content in enumerate(data):
        optimizer.zero_grad()
        print(f'News id: {n_id}')
        name = ast.literal_eval(name)        
        pred = model(cc.convert(content))
        if len(name) is 0:    
            label = torch.Tensor([0]).to(gpu)
        else:
            label = torch.Tensor([1]).to(gpu)
        loss = criterion(x,y) 
        loss.backward()  
        optimizer.step()
        if (i+1) % 100 is 0:
            test_loss = test()
    return best_model

In [ ]:
def main():
    data = pd.load_csv('./train')
    train(data)
    
    

In [ ]:
if __name__ == '__main__':
    main()
